# Module 4: Using a Service to Expose Your App

## Objectives

* Learn about a Service in Kubernetes
* Understand how labels and LabelSelector objects relate to a Service
* Expose an application outside a Kubernetes cluster using a Service

## Overview of Kubernetes Services

Kubernetes [Pods](https://kubernetes.io/docs/concepts/workloads/pods/) are mortal. Pods in fact have a [lifecycle](https://kubernetes.io/docs/concepts/workloads/pods/pod-lifecycle/). When a worker node dies, the Pods running on the Node are also lost. A [ReplicaSet](https://kubernetes.io/docs/concepts/workloads/controllers/replicaset/) might then dynamically drive the cluster back to desired state via creation of new Pods to keep your application running. As another example, consider an image-processing backend with 3 replicas. Those replicas are exchangeable; the front-end system should not care about backend replicas or even if a Pod is lost and recreated. That said, each Pod in a Kubernetes cluster has a unique IP address, even Pods on the same Node, so there needs to be a way of automatically reconciling changes among Pods so that your applications continue to function.

A Service in Kubernetes is an abstraction which defines a logical set of Pods and a policy by which to access them. Services enable a loose coupling between dependent Pods. A Service is defined using YAML [(preferred)](https://kubernetes.io/docs/concepts/configuration/overview/#general-configuration-tips) or JSON, like all Kubernetes objects. The set of Pods targeted by a Service is usually determined by a *LabelSelector* (see below for why you might want a Service without including `selector` in the spec).

Although each Pod has a unique IP address, those IPs are not exposed outside the cluster without a Service. Services allow your applications to receive traffic. Services can be exposed in different ways by specifying a `type` in the ServiceSpec:

* *ClusterIP* (default) - Exposes the Service on an internal IP in the cluster. This type makes the Service only reachable from within the cluster.
* *NodePort* - Exposes the Service on the same port of each selected Node in the cluster using NAT. Makes a Service accessible from outside the cluster using `<NodeIP>:<NodePort>`. Superset of ClusterIP.
* *LoadBalancer* - Creates an external load balancer in the current cloud (if supported) and assigns a fixed, external IP to the Service. Superset of NodePort.
* *ExternalName* - Exposes the Service using an arbitrary name (specified by `externalName` in the spec) by returning a CNAME record with the name. No proxy is used. This type requires v1.7 or higher of `kube-dns`.

More information about the different types of Services can be found in the [Using Source IP](https://kubernetes.io/docs/tutorials/services/source-ip/) tutorial. Also see [Connecting Applications with Services](https://kubernetes.io/docs/concepts/services-networking/connect-applications-service).

Additionally, note that there are some use cases with Services that involve not defining `selector` in the spec. A Service created without `selector` will also not create the corresponding Endpoints object. This allows users to manually map a Service to specific endpoints. Another possibility why there may be no selector is you are strictly using `type: ExternalName`.

>A Kubernetes Service is an abstraction layer which defines a logical set of Pods and enables external traffic exposure, load balancing and service discovery for those Pods.*

### Services and Labels

<img src="https://d33wubrfki0l68.cloudfront.net/cc38b0f3c0fd94e66495e3a4198f2096cdecd3d5/ace10/docs/tutorials/kubernetes-basics/public/images/module_04_services.svg" width=400>

A Service routes traffic across a set of Pods. Services are the abstraction that allow pods to die and replicate in Kubernetes without impacting your application. Discovery and routing among dependent Pods (such as the frontend and backend components in an application) is handled by Kubernetes Services.

Services match a set of Pods using [labels and selectors](https://kubernetes.io/docs/concepts/overview/working-with-objects/labels), a grouping primitive that allows logical operation on objects in Kubernetes. Labels are key/value pairs attached to objects and can be used in any number of ways:

* Designate objects for development, test, and production
* Embed version tags
* Classify an object using tags

<img src="https://d33wubrfki0l68.cloudfront.net/b964c59cdc1979dd4e1904c25f43745564ef6bee/f3351/docs/tutorials/kubernetes-basics/public/images/module_04_labels.svg" width=400>


Labels can be attached to objects at creation time or later on. They can be modified at any time. Let's expose our application now using a Service and apply some labels.

### Lab 4 - Explose your app publicly
#### Step 1 of 2: Create a new service

Let’s verify that our application is running. We’ll use the kubectl get command and look for existing Pods:

In [49]:
studentId=$(grep hpecp-user $HOME/.kube/config | cut -d= -f2)
POD_NAME=$(kubectl get pods -o go-template --template '{{range .items}}{{.metadata.name}}{{"\n"}}{{end}}' | grep $studentId) 
port=$(expr $(echo ${studentId} | tr -cd '[[:digit:]]') + 8000)
kubectl get pods $POD_NAME

NAME                                               READY   STATUS    RESTARTS   AGE
kubernetes-bootcamp-student1125-55cc476c66-gjdr4   1/1     Running   0          5d4h


Next, let’s list the current Services from our cluster:

In [50]:
kubectl get services | grep $studentId

kubernetes-bootcamp-student1125   NodePort    10.107.42.35    <none>        8080:32236/TCP   16m


There is no service yet. To create a new service and expose it to external traffic we’ll use the expose command with NodePort as parameter.

In [33]:
kubectl expose deployment/kubernetes-bootcamp-$studentId --type="NodePort" --port 8080

service/kubernetes-bootcamp-student1125 exposed


Let’s run again the get services command:

In [51]:
kubectl get services

kubernetes-bootcamp-student1125   NodePort    10.107.42.35    <none>        8080:32236/TCP   16m


We have now a running Service called kubernetes-bootcamp-studentId. Here we see that the Service received a unique cluster-IP, an internal port and an external-IP (the IP of the Node).

To find out what port was opened externally (by the NodePort option) we’ll run the describe service command:

In [52]:
kubectl describe services/kubernetes-bootcamp-$studentId

Name:                     kubernetes-bootcamp-student1125
Namespace:                k8shacktenant
Labels:                   app=kubernetes-bootcamp-student1125
Annotations:              <none>
Selector:                 app=kubernetes-bootcamp-student1125
Type:                     NodePort
IP:                       10.107.42.35
Port:                     <unset>  8080/TCP
TargetPort:               8080/TCP
NodePort:                 <unset>  32236/TCP
Endpoints:                10.192.0.144:8080
Session Affinity:         None
External Traffic Policy:  Cluster
Events:                   <none>


Create an environment variable called NODE_PORT that has the value of the Node port assigned.
And another one that compute NODE_NAME from a  `kubectl describe pods ` command. 

In [120]:
NODE_PORT=$(kubectl get services/kubernetes-bootcamp-$studentId -o go-template='{{(index .spec.ports 0).nodePort}}')
echo NODE_PORT=$NODE_PORT
NODE_NAME=$(kubectl describe pods $POD_NAME | grep Node:)
NODE_NAME=${NODE_NAME#Node:} 
NODE_NAME=${NODE_NAME%/*} 
NODE_NAME=$(echo $NODE_NAME | xargs)
echo NODE_NAME=$NODE_NAME


NODE_PORT=32236
NODE_NAME=worker27.etc.fr.comm.hpecorp.net


We can now call the service to show that it's exposed. 

In [122]:
curl http://$NODE_NAME:$NODE_PORT

Hello Kubernetes bootcamp! | Running on: kubernetes-bootcamp-student1125-55cc476c66-gjdr4 | v=1


>The service is exposed through any node from the cluster. You can try this with any of the worker node name part of your cluster, retrieved with ` kubectl get nodes `

#### Step 2 of 2: Using labels

The Deployment created automatically a label for our Pod. With ` describe deployment ` command you can see the name of the labels:

In [133]:
kubectl describe deployment kubernetes-bootcamp-$studentId

Name:                   kubernetes-bootcamp-student1125
Namespace:              k8shacktenant
CreationTimestamp:      Thu, 28 Jan 2021 12:45:22 +0000
Labels:                 app=kubernetes-bootcamp-student1125
Annotations:            deployment.kubernetes.io/revision: 1
Selector:               app=kubernetes-bootcamp-student1125
Replicas:               1 desired | 1 updated | 1 total | 1 available | 0 unavailable
StrategyType:           RollingUpdate
MinReadySeconds:        0
RollingUpdateStrategy:  25% max unavailable, 25% max surge
Pod Template:
  Labels:  app=kubernetes-bootcamp-student1125
  Containers:
   kubernetes-bootcamp:
    Image:        gcr.io/google-samples/kubernetes-bootcamp:v1
    Port:         <none>
    Host Port:    <none>
    Environment:  <none>
    Mounts:       <none>
  Volumes:        <none>
Conditions:
  Type           Status  Reason
  ----           ------  ------
  Available      True    MinimumReplicasAvailable
  Progressing    True    NewReplicaSetAvailable

Let’s use this label to query our list of Pods. We’ll use the kubectl get pods command with -l as a parameter, followed by the label values:

In [134]:
kubectl get pods -l app=kubernetes-bootcamp-$studentId

NAME                                               READY   STATUS    RESTARTS   AGE
kubernetes-bootcamp-student1125-55cc476c66-gjdr4   1/1     Running   0          5d20h


You can do the same to list the existing services:

In [127]:
kubectl get services -l app=kubernetes-bootcamp-$studentId

NAME                              TYPE       CLUSTER-IP     EXTERNAL-IP   PORT(S)          AGE
kubernetes-bootcamp-student1125   NodePort   10.107.42.35   <none>        8080:32236/TCP   16h


To apply a new label we use the label command followed by the object type, object name and the new label:

In [129]:
kubectl label pod $POD_NAME version=v1

pod/kubernetes-bootcamp-student1125-55cc476c66-gjdr4 labeled


This will apply a new label to our Pod (we pinned the application version to the Pod), and we can check it with the describe pod command:

In [130]:
kubectl describe pods $POD_NAME

Name:         kubernetes-bootcamp-student1125-55cc476c66-gjdr4
Namespace:    k8shacktenant
Priority:     0
Node:         worker27.etc.fr.comm.hpecorp.net/16.31.84.69
Start Time:   Thu, 28 Jan 2021 12:44:08 +0000
Labels:       app=kubernetes-bootcamp-student1125
              pod-template-hash=55cc476c66
              version=v1
Annotations:  cni.projectcalico.org/podIP: 10.192.0.144/32
              cni.projectcalico.org/podIPs: 10.192.0.144/32
              kubernetes.io/psp: hcp-psp-privileged
Status:       Running
IP:           10.192.0.144
IPs:
  IP:           10.192.0.144
Controlled By:  ReplicaSet/kubernetes-bootcamp-student1125-55cc476c66
Containers:
  kubernetes-bootcamp:
    Container ID:   docker://2e564c56b2147258c943d83cc6783e74be1e72aa365f84b541d5948c6557235a
    Image:          gcr.io/google-samples/kubernetes-bootcamp:v1
    Image ID:       docker-pullable://gcr.io/google-samples/kubernetes-bootcamp@sha256:0d6b8ee63bb57c5f5b6156f446b3bc3b3c143d233037f3a2f00e279c8fcc64af


We see here that the label is attached now to our Pod. And we can query now the list of pods using the new label:

In [135]:
kubectl get pods -l version=v1

NAME                                               READY   STATUS    RESTARTS   AGE
kubernetes-bootcamp-student1125-55cc476c66-gjdr4   1/1     Running   0          5d20h


This terminates module 4. Let's continue with [module 5, Scale up your app](./6-WKSHP-K8S101-mod5.ipynb).